# N Queen Problem
### The N Queen is the problem of placing N chess queens on an N×N chessboard so that no two queens attack each other
<center><img src="./Images/8queen.png" class="bg-primary mb-3"></center><br>


In [22]:
import random
from IPython.display import Image

### 1) Generate initial population (n random chromosome)
Suppose that each queen is located in each column of chess board. Thus, every digit in a chromosome shows the location of queen in a column. For example, [2,3,6,8,5,4,2,7] shows a 8*8 chess board that queen in column 3 is located n row 6.  **Size** in this function shows the number of chromosomes produced in this step.
<center><img src="./Images/chromosome.png" class="bg-primary mb-3"></center><br>


In [23]:
def random_chromosome(size):  # making random chromosomes
    return [random.randint(1, size) for _ in range(size)]

### 2) Find the value of fitness function for each chromosome
**Fitness function**: the number of nonattacking pairs of queens

Since that we supposed that each queen i located in each column, there is no possibility to see attcking queen in a column. Hence, we must check whether a quuen aatcks any piece in the same row or diagonal.

1) &emsp;To compute collision in a row,the only thing we have to do is to look for the same number in the chromosomes.
2) &emsp;To compute collision in diagonals, there are twoo methods:<br>
&emsp;&emsp;(a)&emsp;For a simple method of finding conflicts [4], consider an ntuple: $(q_{1},..., q_{i},..., q_{j}, ..., q_{n})$. i-th and j-th queen share a
diagonal if:<br>
$$ i - q_{i} = j - q_{j}$$   
$$or$$   
$$i + q_{i} = j + q_{j}$$
&emsp;&emsp;&emsp;which reduces to:
$$ |q_{i} - q_{j}| = |i - j|$$
&emsp;&emsp;&emsp;This simple approach results in fitness function with complexity of $O(n^{2})$. It is possible to reduce complexity to $O(n)$ by observing diagonals on the board as the following.
&emsp;&emsp;(b)&emsp; Left and right are defined in this method, Fig. 1. There are $2n-1$ "left" (left to right) and $2n-1$ "right" (right to left) diagonals.<br>
&emsp;&emsp; In this situation, there is no need to compute nested loop and just need to compute the number of queens in each diagonal. Thus, fitness function is of order $ O(n^{2})$.
<center><img src="./Images/Left.png" class="bg-primary mb-3"><img src="./Images/Right.png" class="bg-primary mb-3" ></center><br>


In [24]:
#  Method (b)
def fitness(chromosome):
    horizontal_collisions = sum(
        [chromosome.count(queen)-1 for queen in chromosome])/2
    diagonal_collisions = 0

    n = len(chromosome)
    left_diagonal = [0] * 2*n
    right_diagonal = [0] * 2*n
    for i in range(n):
        left_diagonal[i + chromosome[i] - 1] += 1
        right_diagonal[len(chromosome) - i + chromosome[i] - 2] += 1

    diagonal_collisions = 0
    for i in range(2*n-1):
        counter = 0
        if left_diagonal[i] > 1:
            counter += left_diagonal[i]*(left_diagonal[i]-1)/2
        if right_diagonal[i] > 1:
            counter += right_diagonal[i]*(right_diagonal[i]-1)/2
        diagonal_collisions += counter  # / (n-abs(i-n+1))
    # 28-(2+3)=23
    return int(maxFitness - (horizontal_collisions + diagonal_collisions))


# Method (a)
def fitness1(chromosome):
    horizontal_collisions = sum(
    [chromosome.count(queen)-1 for queen in chromosome])/2
    diagonal_collisions = 0

    n = len(chromosome)
    for i in range(n):

        for j in range(i + 1, n):
            if (abs(i - j) == abs(chromosome[i] - chromosome[j])) and (chromosome[i] != chromosome[j]):
                diagonal_collisions += 1
    return int(maxFitness-(horizontal_collisions + diagonal_collisions))

### 3) Sorting chromosome based on **Fitness function**
&emsp; Sorting population and eliminate the weak chromosomes 

In [25]:
def survival(population, fitness):
    fitnessPopulation = [fitness(n) for n in population]
    populationWithHighFitness = zip(population, fitnessPopulation)
    populationWithHighFitness = list(populationWithHighFitness)

    sortedPopulation = sorted(
        populationWithHighFitness, key=lambda x: x[1], reverse=True)
    
    survivalPopolation = []
    for i in range(int(len(sortedPopulation)/2)):
        
        survivalPopolation.append(sortedPopulation[i][0])
        # print(sortedPopulation)
    return survivalPopolation


### 4) Select mates based on  **Roulette Wheel weighting**
&emsp; a) The probability of selection is calculated from the fitness function of the chromosome rather. 

In [26]:
def probability(chromosome, fitness):
    return fitness(chromosome) / maxFitness

&emsp; b) Random function: A random number between zero and one is generated. 

In [27]:
def random_pick(population, probabilities):
    populationWithProbabilty = zip(population, probabilities)
    total = sum(w for c, w in populationWithProbabilty)
    r = random.uniform(0, total)
    upto = 0
    for c, w in zip(population, probabilities):
        if upto + w >= r:
            return c
        upto += w
    assert False, "Shouldn't get here"


### 5) Mating: 
Mating is the creation of one or more offspring from the parents selected in the pairing process

In [28]:
def mating(x, y):  # doing cross_over between two chromosomes
    n = len(x)
    c = random.randint(0, n - 1)
    return x[0:c] + y[c:n]


### 6) Mutation:
Random mutations alter a certain percentage of the bits in the list of chromosomes. In this problem 


In [29]:
def mutate(x):  # randomly changing the value of a random index of a chromosome
    n = len(x)
    c = random.randint(0, n - 1)
    m = random.randint(1, n)
    x[c] = m
    return x


## Genetic Algorithm

In [30]:
def genetic_algorithm(population, fitness):
    mutation_probability = 0.06
    best_population = survival(population, fitness)
    probabilities = [probability(n, fitness) for n in best_population]
    nn = len(best_population)
    for i in range(nn):
        x = random_pick(population, probabilities)  # best chromosome 1
        y = random_pick(population, probabilities)  # best chromosome 2
        # creating two new chromosomes from the best 2 chromosomes
        child = mating(x, y)
        if (random.random() < mutation_probability): 
            child = mutate(child)
            if i != 0 :   # Elitism
                best_population[i] = mutate(best_population[i])
            
        print_chromosome(child)
        best_population.append(child)
        # print(best_population)
        if fitness(child) == maxFitness:
            break
    return best_population


In [31]:
def print_chromosome(chrom):
    print("Chromosome = {},  Fitness = {}"
          .format(str(chrom), fitness(chrom)))

In [32]:
if __name__ == "__main__":
    nq = int(input("Enter Number of Queens: "))  # say N = 8
    maxFitness = (nq*(nq-1))/2  # n (n - 1)/2 = 8 * 7 / 2 = 28
    population = [random_chromosome(nq) for _ in range(10)]
    generation = 1

    while not maxFitness in [fitness(chrom) for chrom in population]:
        print("=== Generation {} ===".format(generation))
        population = genetic_algorithm(population, fitness)
        print("")
        print("Maximum Fitness = {}".format(
            max([fitness(n) for n in population])))
        generation += 1
    chrom_out = []
    print("Solved in Generation {}!".format(generation-1))
    for chrom in population:
        if fitness(chrom) == maxFitness:
            print("")
            print("One of the solutions: ")
            chrom_out = chrom
            print_chromosome(chrom)

    board = []

    for x in range(nq):
        board.append(["x"] * nq)

    for i in range(nq):
        board[nq-chrom_out[i]][i] = "Q"

    def print_board(board):
        for row in board:
            print(" ".join(row))

    print()
    print_board(board)


=== Generation 1 ===
Chromosome = [1, 6, 8, 4, 2, 4, 8, 1],  Fitness = 23
Chromosome = [6, 1, 7, 5, 4, 3, 8, 2],  Fitness = 22
Chromosome = [6, 1, 7, 2, 3, 5, 5, 8],  Fitness = 23
Chromosome = [8, 3, 5, 6, 7, 4, 6, 8],  Fitness = 22
Chromosome = [1, 6, 8, 4, 2, 4, 8, 1],  Fitness = 23

Maximum Fitness = 23
=== Generation 2 ===
Chromosome = [4, 5, 3, 2, 7, 3, 5, 8],  Fitness = 21
Chromosome = [1, 1, 7, 5, 4, 3, 8, 2],  Fitness = 21
Chromosome = [6, 5, 1, 8, 1, 5, 3, 2],  Fitness = 23
Chromosome = [6, 3, 5, 6, 1, 5, 3, 2],  Fitness = 20
Chromosome = [1, 6, 1, 8, 1, 5, 3, 2],  Fitness = 22

Maximum Fitness = 23
=== Generation 3 ===
Chromosome = [6, 1, 7, 2, 3, 4, 8, 1],  Fitness = 23
Chromosome = [6, 1, 7, 2, 3, 5, 5, 8],  Fitness = 23
Chromosome = [6, 1, 7, 2, 3, 4, 8, 1],  Fitness = 23
Chromosome = [1, 6, 8, 4, 2, 4, 5, 8],  Fitness = 21
Chromosome = [6, 1, 7, 2, 3, 5, 8, 1],  Fitness = 25

Maximum Fitness = 25
=== Generation 4 ===
Chromosome = [6, 1, 7, 2, 3, 5, 3, 2],  Fitness = 21
Ch

Ref: Solving n-Queen problem using global parallel genetic algorithm Extended Abstract
Marko Božikovi, Marin Golub, Leo Budin